## 准备数据

In [27]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [28]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [29]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 第一层参数：输入层(784) -> 隐藏层(128)
        self.W1 = tf.Variable(tf.random.normal([784, 128], stddev=0.1), name='W1', trainable=True)
        self.b1 = tf.Variable(tf.zeros([128]), name='b1', trainable=True)
        
        # 第二层参数：隐藏层(128) -> 输出层(10)
        self.W2 = tf.Variable(tf.random.normal([128, 10], stddev=0.1), name='W2', trainable=True)
        self.b2 = tf.Variable(tf.zeros([10]), name='b2', trainable=True)
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])            # 将输入从 [batch, 28, 28] 展平成 [batch, 784]
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)   # 第一层：全连接 + ReLU激活
        logits = tf.matmul(h1, self.W2) + self.b2          # 第二层：全连接（未激活）
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [30]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [31]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.448026 ; accuracy 0.13041666
epoch 1 : loss 2.412963 ; accuracy 0.13513333
epoch 2 : loss 2.3820648 ; accuracy 0.13981667
epoch 3 : loss 2.354535 ; accuracy 0.14511667
epoch 4 : loss 2.329753 ; accuracy 0.1503
epoch 5 : loss 2.307238 ; accuracy 0.15573333
epoch 6 : loss 2.2866075 ; accuracy 0.16141666
epoch 7 : loss 2.2675593 ; accuracy 0.16736667
epoch 8 : loss 2.24985 ; accuracy 0.17358333
epoch 9 : loss 2.2332845 ; accuracy 0.17953333
epoch 10 : loss 2.2177007 ; accuracy 0.18571667
epoch 11 : loss 2.2029636 ; accuracy 0.19248334
epoch 12 : loss 2.1889615 ; accuracy 0.19956666
epoch 13 : loss 2.1756024 ; accuracy 0.2066
epoch 14 : loss 2.1628075 ; accuracy 0.21303333
epoch 15 : loss 2.150514 ; accuracy 0.22045
epoch 16 : loss 2.1386619 ; accuracy 0.22741666
epoch 17 : loss 2.1272066 ; accuracy 0.23448333
epoch 18 : loss 2.1161077 ; accuracy 0.24216667
epoch 19 : loss 2.1053288 ; accuracy 0.24946667
epoch 20 : loss 2.0948372 ; accuracy 0.2566
epoch 21 : loss 2.0846064